# language tools

Converting between languages, parsing languages computer languages, demonstrating tools like syntax hilighters, and outputting the diffed code


## introduction



TODO: use syntax tree to convert from and to any language


use a common library for language generation 

https://bitbucket.org/birkenfeld/pygments-main/src/default/pygments/lexers/haskell.py

https://github.com/PrismJS/prism/blob/master/components/prism-markdown.js

http://qbnz.com/highlighter/

https://stackoverflow.com/questions/171924/how-do-i-syntax-check-a-bash-script-without-running-it

bash -n scriptname

Improve on the babel-preset-php pattern of replacing code with a tree, just use templates and the tree creator on regular code blocks

https://gitlab.com/kornelski/babel-preset-php/blob/master/lib/trans.js

```
function translateAssignBlock(node, scope) {
    if (isVariableCreatingAssignment(node, scope)) {
        return makeVariableDeclaration(node.left.name, "var", translateExpr(node.right, scope), node.loc, scope);
    }
    return wrapInStatement(translateAssign(node, scope));
}
```

should be something like:
```
var ASSIGMENT_EXPRESSION = (leftName, right) => {
    var leftName = right;
}

function translateAssignBlock(node, scope) {
    if(//Assignment[/Identifier].includes(node)) {
        return getAST(ASSIGMENT_EXPRESSION, node)
    }
    return getAST(STATEMENT, node)
}
```

that way when they are written in multiple languages we can transpile any language, maybe

use this library as a reference for lots of languages https://github.com/seaneshbaugh/rosetta-euler

automatically translate https://projecteuler.net/archives to any language


TODO: use babel and a transpiling kernel from babel-preset-php, 

TODO: make a generic translator that does manipulations on queries based on templates






In [ ]:
// sample definition for MatLab



In [ ]:
// definition for nodejs



## syntax tree tools



### justification

Just so it can be parsed with XPath instead. Get them all working the same!




Crap modules I won't need any more:
- css
- ASTQ
- Acorn
- esprima
- escodegen
- esquery
- jsonpath
- jsonquery
- jspath
- json:select
- jsel

TODO: get rid of jsonpath-plus, jsonstream, jspath, 



```
/* expected output `
[ Script {
    type: 'Program',
    body: [ [FunctionDeclaration], [ExpressionStatement] ],
    sourceType: 'script',
    range: [ 1, 45 ],
    comments: [],
    tokens:

<program type="program">
    <function>
    </function>
    <expression>
    </expression>
</program>

*/
```



### convert a syntax tree to html?

Reasoning. Every atom is made up of smaller component parts. This conversion creates a tree that breaks down to &lt;elements&gt; and attr="ibutes", just like neutrons and protons.


In [ ]:

function statementToElement(statement) {
    var {type} = statement;
    var result = ``, attrs = ``;
    var isList = true;
    for(var i in statement) {
        if(parseInt(i) + '' === i + '') {
            result += statementToElement(statement[i])
            continue;
        }
        isList = false;
        var jsType = typeof statement[i];
        if(jsType === 'object' && statement[i]) {
            jsType = Object.getPrototypeOf(statement[i]).constructor.name;
        }
        // if the property is an Object, print out as a child
        // TODO: replace this type property when converting other trees
        //   should be like getEl() statementToElement(() => el.type)
        if(typeof statement[i].type !== 'undefined'
           || (typeof statement[i][0] !== 'undefined'
        // or if the property is an Array, print them out as child elements
           && typeof statement[i][0].type !== 'undefined')) {
            // print out the other keys just for fun matching
            attrs +=  `
${i}="" ${i}-type="${jsType}"`;
            result += `<${i}-attr>${statementToElement(statement[i])}</${i}-attr>`
        } else {
            // if the property is not an object, i.e. native types
            attrs +=  `
${i}="${statement[i]}" ${i}-type="${jsType}"`
        }
    }
    
    return isList ? result : `
<${type}${attrs}>${result}</${type}>`
}

module.exports = statementToElement;


### convert an html tree to objects?



In [ ]:
var {JSDOM} = require('jsdom');

// TODO: create a pattern same as notebook markdown in core, accumulate
function accumulateComments(nodes) {
    var commentBuffer = []
    return nodes.reduce((acc, n) => {
        if(typeof n !== 'object') {
            commentBuffer.push[n];
        } else {
            n.comments = commentBuffer;
            commentBuffer = [];
            acc.push(n);
        }
        return acc;
    }, []);
}

// expects a string or a tree from JSDOM
function elementToStatement(body) {
    if(typeof body === 'string') {
        var dom = new JSDOM(body);
        body = dom.window.document.body;
    }
    if(body.nodeName === '#text') {
        return body.nodeValue;
    }
    var attrs = body.getAttributeNames()
        .filter(a => a.substr(-5) !== '-type');
    if(attrs.length === 0) {
        var elsAndText = Array.from(body.childNodes)
            .map(c => elementToStatement(c));
        return accumulateComments(elsAndText);
    }
    var result = attrs.reduce((obj, p) => {
        var childObject = Array.from(body.childNodes)
           .filter(n => n.nodeName.toLowerCase() === p.toLowerCase() + '-attr')[0];
        var type = body.getAttribute(p + '-type');
        if(childObject) {
            // converted from an array of childNodes above
            var children = elementToStatement(childObject);
            obj[p] = type === 'Array' ? children : children[0];
        } else {
            var attr = body.getAttribute(p);
            obj[p] = attr;
            if(type === 'boolean')  obj[p] = attr === 'true';
            if(type === 'number')  obj[p] = parseInt(attr);
        }
        return obj;
    }, {})
    return result;
}

module.exports = elementToStatement;



#### test tree to html?

In [ ]:
var importer = require('../Core');
var assert = require('assert');
// TODO: make this dependency obsolete
var esquery = require('esquery');
var esprima = require('esprima'); 
var escodegen = require('escodegen');
var ASTQ  = require("astq");
var astq = new ASTQ();

var gen_options = {
    comment: true,
    tokens: true,
    whitespace: true,
    format: {indent: {style: '    '}, quotes: 'single'}
}

function setupASTArray(code) {
    var ctx = esprima.parse(code, {range: true, tokens: true,
                        comment: true, whitespace: true})
    
    escodegen.attachComments(ctx, ctx.comments, ctx.tokens, ctx.whitespace);
    return ctx;
}

function testASTArray(ctx) {
    var output1 = escodegen.generate(ctx, gen_options)
    console.log(output1);
    var page = statementToElement(ctx);
    var translated_back = elementToStatement(page)[0];
    var output2 = escodegen.generate(translated_back, gen_options);
    assert(output1 === output2, 'different code trees');    
    return page;
}

module.exports = testASTArray;

function htmlEntities(str) {
    return String(str)
        .replace(/&/g, '&amp;')
        .replace(/</g, '&lt;')
        .replace(/>/g, '&gt;')
        .replace(/"/g, '&quot;');
}

if(typeof $$ !== 'undefined') {
    var code = `
function name(params) {
    
}
console.log()
`
    var ctx = setupASTArray(code);
    var html = testASTArray(ctx)
    console.log(astq.query(ctx, '*'));
    console.log(astq.query(ctx, '*')[0].tokens);
    console.log(esquery(ctx, '*').length);
    console.log(esquery(ctx, 'Program').length);
    
    $$.html('<pre>' + htmlEntities(html) + '</pre>')
}


expected output


```
FunctionDeclaration {
    type: 'FunctionDeclaration',
    id:
     Identifier { type: 'Identifier', name: 'name', range: [Array] },
    params: [ [Identifier] ],
    body:
     BlockStatement { type: 'BlockStatement', body: [], range: [Array] },
    generator: false,
    expression: false,
    async: false,
    range: [ 1, 31 ] },
  ExpressionStatement {
    type: 'ExpressionStatement',
    expression:
     CallExpression {
       type: 'CallExpression',
       callee: [StaticMemberExpression],
       arguments: [],
       range: [Array] },
    range: [ 32, 45 ] } ]
```


### improved syntax manipulation

replacing that functionality I noticed in babel somewhere



#### test 0

Pretty nice syntax but does it work on non, C-style languages.

TODO: derive from the names and how the ```require()``` is used and automatically fix ```{expanded}``` or ```assignment =``` imports.



In [62]:

function transpile(definition) {
    console.log('hit');
}

var phpToJs = [
    [() => array_shift(arr), () => arr.shift()],
    [() => strpos(str, match), () => { // has parameters so call this function instead?
        var i = str.indexOf(match);
        return i > -1 ? i : false}],
    [() => substr(str, start, length), () => str.substr(start, length)],
    [() => is_numeric(num), () => !isNaN(parseInt(num)) || !isNaN(parseFloat(num))],
    [() => abs(), () => Math.abs()], // doesn't mess with parameters
    // ...call(['abs', 'sin', 'cos', 'pow'], (id) => Math.id()))
    [['abs', 'sin', 'cos', 'pow', 'floor', 'ceil', 'round', 'max', 'min'], {
        `//*[contains(@type, "Call")`: (id) => Math.id(), // optional ()
    }]
]

// this will work as long as PHP syntax is also called a Call or Member or Indentifier
var jsToPhp = phpToJs.map(d => [d[1], d[0]])

function replaceImport() {
    return transpile(
        (str = (({ctx, str}) => {
            console.log('do some manipulation on the ctx of str');
            console.log(str);
            console.log(this);
        })) => importer.import(str),
        (ctx = ({ctx}) => {
            // default contstructor serves to do manipulation on matching parent ctx
            console.log(ctx);
        }) => require(str)
    );
}

function replaceCore() {
    transpile(
        () => {
            var importer = require('../Core');
        },
        () => {}
    )
    transpile(
        () => importer.callee,
        ({callee = ({ctx, callee}) => {
            if(callee.name === 'import') callee.name = 'importNotebook'
        }, ctx}) => ctx.callee = callee.property
    )
}



SyntaxError: Unexpected token var

#### test xpath based transpiling

A different style of syntax manipulation to reduce amount of code needed in babel.

This method would replace the one in the notebook export script.



In [ ]:
// this is pretty nice

function transpile(definition) {
}


var CORE_REQUIRE = `/*[@type="Literal" and contains(@value, "Core")]`;
var CORE_DECLARE = `//*[contains(@type, "Variable") and contains(., ${CORE_REQUIRE})]`;
var IMPORTER = `${CORE_DECLARE}/*[@type="Identifier"]/@name`;
var IMPORTER_CALLS = `//*[contains(@type, "Call") and ./*[
    contains(@type, "Member") and ./*[@type="Identifier" and @name=${IMPORTER}]
]]`;

function replaceCore(ast) {
    var impName = query(ast, IMPORTER)[0];
    return transpile({
        CORE_DECLARE: '',
        IMPORTER_CALLS: (parent) => {
            parent.callee = parent.callee.property
            if(parent.callee.name === 'import') parent.callee.name = 'importNotebook'
        }
    })
}


var esquery = require('esquery');
var jsel = require('jsel');
var esprima = require('esprima'); 
var escodegen = require('escodegen');
var cheerio = require('cheerio')
var {XPathResult, JSDOM} = require('jsdom');

var getArrayXPath = (selector, ctx) => {
    var iterator = (ctx.evaluate || ctx.ownerDocument.evaluate)(
        selector,
        ctx, null,
        (XPathResult || {}).ORDERED_NODE_ITERATOR_TYPE || 5, null);
    var co = [];
    var m;
    while ((m = iterator.iterateNext()) && co.push(m.nodeValue || m)) ;
    return co;
};

// replace core
function testReplaceCore(code) {
    var ast = esprima.parse(code, {range: true, tokens: true,
                            comment: true, whitespace: true})
    var body = jsel(ast);
    //var funcOut1 = body.selectAll(`//*[@type="FunctionDeclaration"]`);
    var code = body.selectAll(IMPORTER_CALLS);
    console.log(code);
    
    translated_back = code[0];
    // TODO: test the same exact statements on JSDOM with our implementation
    var page = statementToElement(ast);
    var dom = new JSDOM(page);
    var els = getArrayXPath(CORE_DECLARE, dom.window.document.body);
    var translated_back = elementToStatement(els[0]);
    return translated_back;
}


var code = `
var importer = require('../Core');
function name(params) {
    return importer.interpret('this is a describe request');
}
console.log()
`

if(typeof $$ !== 'undefined') {
    var translated_back = testReplaceCore(code);
    var output2 = escodegen.generate(translated_back);
    console.log(output2);
    // TODO: test with CSS style
    //""`*[type*='StaticMemberExpression'] > *[name=="importer"]`;
    //var $ = cheerio.load(page);
}



#### TODO: convert between astq and other tree libraries



In [ ]:

var keywords = ['type', 'id', 'params',
                'body', 'generator', 'async',
                'range', 'expression', 'callee',
                'arguments'];


#### TODO: convert trees using this library



In [ ]:
const fs = require('mz/fs');
const xpath = require('xpath');
const parse5 = require('parse5');
const xmlser = require('xmlserializer');
const dom = require('xmldom').DOMParser;

(async () => {
    const html = await fs.readFile('./test.htm');
    const document = parse5.parse(html.toString());
    const xhtml = xmlser.serializeToString(document);
    const doc = new dom().parseFromString(xhtml);
    const select = xpath.useNamespaces({"x": "http://www.w3.org/1999/xhtml"});
    const nodes = select("//x:a/@href", doc);
    console.log(nodes);
})();



### convert between languages

Improve on the babel-preset-php pattern of replacing code with a tree, just use templates and the tree creator on regular code blocks

https://gitlab.com/kornelski/babel-preset-php/blob/master/lib/trans.js



In [ ]:

// make sure this format works both directions
// from PHP to javascript
var fromPhpToJavascropt = {
    '//CallExpression[@name="array_shift"]': {
        
    }
}

var fromJavascriptToPhp = {
    
}


#### translation formats test

Should only need one description format from notebook module exporter to improve


In [ ]:
// from selenium notebooks
    .getAllUntil(
        '//*[contains(@role, "main")]//*[contains(@class, "scrollable")][contains(.,"Messages")]',
        '//*[contains(@role, "main")]//*[contains(@class, "scrollable")][contains(.,"Messages")]//h4//text()',
        [],
        (a, b) => a + '' === b + '',
        i => i < 10,
        true /* scroll up, not down */
    )
    .getAllXPath([
        '//*[contains(@role, "main")]//*[contains(@class, "scrollable")][contains(.,"Messages")]//h4',
        {
            time: './/text()',
            messages: [
                './following-sibling::div[count(./preceding-sibling::h4)=$i+1]',
                {
                    from: './/h5//text()',
                    message: './/*[not(name()="H5")]/text()'
                }
            ]
        }
    ])

function replaceCore(ast) {
    var impParent = getArrayAST(GET_PARENT(GET_CORE), ast)[0];
    if(impParent) {
        var impDeclare = getArrayAST(GET_CORE, impParent)[0];
        var impName = getArrayAST(GET_CORENAME, impDeclare)[0].name;
        impParent.splice(impParent.indexOf(impDeclare), 1);
        var impUsage = getArrayAST(GET_CALLER(impName), ast);
        impUsage.forEach(parent => {
            parent.callee = parent.callee.property;
            if(parent.callee.name === 'import') parent.callee.name = 'importNotebook'
        });
    }
}


// format from syntax notebook, too much code!!
var GET_CALLED = fnName => `//CallExpression/Identifier[@name == '${fnName}']`;
var GET_IMPORTER = importer => `/*[(@type == 'StaticMemberExpression' || @type == 'MemberExpression')
                && /Identifier[@name == '${importer}']]`;
var GET_CALLER = importer => `//CallExpression[${GET_IMPORTER(importer)}]`;
var GET_PARENT = of => `//*[${of}]`;
// TODO: use form code analysis
var GET_IMPORT_IDENTIFIER = `/Identifier[@name=='import' || @name=='importNotebook']`;
var GET_IMPORT = match => `//CallExpression[
    (${GET_IMPORT_IDENTIFIER}
    || /StaticMemberExpression${GET_IMPORT_IDENTIFIER})
    && //Literal[@value == '${match}']]`;
var GET_EXPORTS = `//AssignmentExpression[//Identifier[@name=='exports']]`;
var GET_FUNCTIONS = `//FunctionDeclaration`;
var GET_IMPORTS = `//CallExpression[${GET_IMPORT_IDENTIFIER} 
    || /StaticMemberExpression${GET_IMPORT_IDENTIFIER}]//Literal`;
var GET_IMPORTER_CALLS = `//CallExpression//Identifier[
    ${[ 'import',
  'streamJson',
  'regexToArray',
  'getCells',
  'runAllPromises',
  'interpret',
  'runInNewContext',
  'cacheCells',
  'interpretMarkdown',
  'fuseSearch',
  'cacheAll' ].map(k => `@name=='${k}'`).join(' || ')}]`

// from javascript to PHP

// from source or from matcher

// format I used on study sauce templates

var array_shift = function (arr) {return arr.shift();};
var strpos = function (str, match) {var i; return (i = str.indexOf(match)) > -1 ? i : false;};
var substr = function (str, start, length) {return str.substr(start, length);};
var is_numeric = function (num) {return !isNaN(parseInt(num)) || !isNaN(parseFloat(num));};
var strlen = function (str) {return (''+(str || '')).length;};
var array_merge = function () {
var isObject = typeof arguments[0] == 'object' && arguments[0] != null && arguments[0].constructor != Array;
var args = [isObject ? {} : []];
for(var a = 0; a < arguments.length; a++) {
    args[args.length] = arguments[a];
};
return args.reduce(function (a, b) {
    return isObject ? $.extend(a, b) : $.merge(a, b);
});
};
var round = function (num, digits) {
    if(digits > 0) {
        return Math.round(num * (10 * (digits ? digits : 0))) / (10 * (digits ? digits : 0));
    }
    return Math.round(num);
};
var is_string = function (obj) {return typeof obj == 'string';};
var is_a = function (obj, typeStr) { return typeof obj == 'object' && obj != null && obj.constructor.name == typeStr();};
var intval = function (str) {var result = parseInt(str); return isNaN(result) ? 0 : result;};
var trim = function (str) {return (str || '').trim();};
var explode = function (del, str) {return (str || '').split(del);};
var array_slice = function (arr, start, length) { return (arr || []).slice(start, length); };
var array_splice = function (arr, start, length) { return arr.splice(start, length); };
var array_search = function (item, arr) { var index = (arr || []).indexOf(item); return index == -1 ? false : index; };
var count = function (arr) { return (arr || []).length; };
var in_array = function (needle, arr) { return (arr || []).indexOf(needle) > -1; };
var array_values = function (arr) { return (arr || []).slice(0); };
var is_array = function (obj) { return typeof obj == 'array' || typeof obj == 'object'; }; // PHP and javascript don't make a distinction between arrays and objects syntax wise using [property], all php objects should be restful anyways
var array_keys = function (obj) {var result=[]; for (var k in obj) { if (obj.hasOwnProperty(k)) { result[result.length] = k } } return result; };
var implode = function (sep, arr) {return (arr || []).join(sep);};
var preg_replace = function (needle, replacement, subject) {
    return (subject || '').replace(new RegExp(needle.split('/').slice(1, -1).join('/'), needle.split('/').slice(-1)[0] + 'g'), replacement);
};
var number_format = function (num, digits) { return parseFloat(num).toFixed(digits);};
var preg_match = function (needle, subject, matches) {
    var result = (new RegExp(needle.split('/').slice(1, -1).join('/'), needle.split('/').slice(-1)[0] + 'g')).exec(subject);
    if(result == null) {
        return 0;
    }
    if(typeof matches != 'undefined') {
        for(var m = 0; m < result.length; m++) {
            matches[m] = result[m];
        }
    }
    return result.length;
};
var ucfirst = function (str) {return (str || '').substr(0, 1).toLocaleUpperCase() + str.substr(1);};
var str_replace = function (needle, replacement, haystack) {return (haystack || '').replace(new RegExp(RegExp.escape(needle), 'g'), replacement);};
var call_user_func_array = function (context, params) {return context[0][context[1]].apply(context[0], params);};
var print = function (s) { window.views.__output += s };
var strtolower = function(s) { return (s || '').toLocaleLowerCase(); };
var empty = function(s) {
    if(typeof s == 'undefined' || ('' + s).trim() == ''
        || s === false || s === 'false' || s == null || s === 0
        || (typeof s == 'object' && s.constructor == Array && s.length == 0)) {
        return true;
    }
    if(typeof s == 'object') {
        for(var p in s) {
            if(s.hasOwnProperty(p)) {
                return false;
            }
        }
    }
};
var json_encode = JSON.stringify;
var method_exists = function (s,m) { return typeof s == 'object' && typeof s[m] == 'function'; };
var isset = function (s) { return typeof s != 'undefined'; };



### generate a synax tree from documentation


#### test language translator

TODO: Creating an interface like this becomes a simple http request to download and create a language compatible pattern, do in C# and in Node




In [ ]:

var wireKernelInterface = {
    ...metaKernelInterface,
    // implement all requests, these are required by the meta kernel
    execute_request, inspect_request, complete_request,
    history_request, is_complete_request, shutdown_request,
    // requests maybe not required right now?
    kernel_info_request, interrupt_request, input_request,
    connect_request, comm_info_request, kernel_info_request,

    // implement all replys
    execute_reply, inspect_reply, complete_reply,
    history_reply, kernel_info_reply, is_complete_reply,
    connect_reply, comm_info_reply, kernel_info_reply,
    shutdown_reply, interrupt_reply, input_reply,

    // a few extra protocol methods
    display_data, update_display_data, execute_input,
    execute_result, error, status, clear_output,
    comm_msg, comm_close
}

module.exports = wireKernelInterface;
    

## TODO: babel transpiler plugin

